In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

import folium
from folium import Choropleth

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../bin/data_preprocess.csv')

In [4]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
33,co2,207,56.208236,56.208236,float64
19,consumo_urbano,142,56.161616,56.161616,float64
18,consumo_carretera,82,56.161616,56.161616,float64
17,consumo_medio,114,50.458430,50.458430,float64
35,id_sobrealimentacion,5,34.079254,43.045843,object
22,cilindrada,199,28.189588,28.189588,float64
20,deposito,81,24.413364,24.413364,float64
34,num_cilindros,7,23.512044,46.511267,float64
12,capacidad_maletero,359,19.906760,19.906760,float64
29,velocidad_max,137,19.704740,19.704740,float64


In [7]:
comunidades_dict = {
    "Andalucía": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"],
    "Aragón": ["Huesca", "Teruel", "Zaragoza"],
    "Asturias": ["Asturias"],
    "Islas Baleares": ["Islas Baleares"],
    "Canarias": ["Las Palmas", "Tenerife"],
    "Cantabria": ["Cantabria"],
    "Castilla-La Mancha": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"],
    "Castilla y León": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"],
    "Cataluña": ["Barcelona", "Girona", "Lleida", "Tarragona"],
    "Extremadura": ["Badajoz", "Cáceres"],
    "Galicia": ["La Coruña", "Lugo", "Orense", "Pontevedra"],
    "Madrid": ["Madrid"],
    "Murcia": ["Murcia"],
    "Navarra": ["Navarra"],
    "La Rioja": ["La Rioja"],
    "País Vasco": ["Álava", "Guipúzcoa", "Vizcaya"],
    "Valencia": ["Alicante", "Castellón", "Valencia"],
    "Ceuta": ["Ceuta"],
    "Melilla": ["Melilla"]
}

In [9]:
provincias_to_comunidades = {
    provincia: comunidad
    for comunidad, provincias in comunidades_dict.items()
    for provincia in provincias
}

In [11]:
listado_provincias = {}
index = 1
for detalle in df['id_provincia'].unique():
    if pd.notnull(detalle): 
        listado_provincias[index] = detalle
        index += 1

df_provincias = pd.DataFrame.from_dict(listado_provincias, orient='index', columns=['nombre_provincia']).reset_index()
df_provincias.rename(columns={'index': 'id_provincia'}, inplace=True)
df_provincias['comunidad_autonoma'] = df_provincias['nombre_provincia'].map(provincias_to_comunidades)
df_provincias.to_csv('../bin/listado_provincias.csv', index=False)

conversion_provincias = {v: k for k, v in listado_provincias.items()}
df['id_provincia'] = df['id_provincia'].map(conversion_provincias)

In [13]:
listado_concesionarios = {}
index = 1
for detalle in df['id_concesionario'].unique():
    if pd.notnull(detalle): 
        listado_concesionarios[index] = detalle
        index += 1

df_concesionarios = pd.DataFrame.from_dict(listado_concesionarios, orient='index', columns=['nombre_concesionario']).reset_index()
df_concesionarios.rename(columns={'index': 'id_concesionario'}, inplace=True)
df_concesionarios.to_csv('../bin/listado_concesionarios.csv', index=False)

conversion = {v: k for k, v in listado_concesionarios.items()}
df['id_concesionario'] = df['id_concesionario'].map(conversion)

In [15]:
listado_modelos = {}
index = 1
for detalle in df['id_modelo'].unique():
    if pd.notnull(detalle): 
        listado_modelos[index] = detalle
        index += 1

df_modelos = pd.DataFrame.from_dict(listado_modelos, orient='index', columns=['nombre_modelo']).reset_index()
df_modelos.rename(columns={'index': 'id_modelo'}, inplace=True)
df_modelos.to_csv('../bin/listado_modelos.csv', index=False)

conversion = {v: k for k, v in listado_modelos.items()}
df['id_modelo'] = df['id_modelo'].map(conversion)

In [17]:
marca_df = pd.read_csv('../bin/listado_marcas.csv')

marca_df['nombre_marca'] = marca_df['nombre_marca'].str.upper()
listado_marcas = marca_df.set_index('id_marca')['nombre_marca'].to_dict()

conversion = {v: k for k, v in listado_marcas.items()}
df['id_marca'] = df['id_marca'].map(conversion)

In [19]:
listado_distintivo_ambiental = {}
index = 1
for detalle in df['id_distintivo_ambiental'].unique():
    if pd.notnull(detalle): 
        listado_distintivo_ambiental[index] = detalle
        index += 1
    
df_distintivo_ambiental = pd.DataFrame.from_dict(listado_distintivo_ambiental, orient='index', columns=['nombre_distintivo']).reset_index()
df_distintivo_ambiental.rename(columns={'index': 'id_distintivo'}, inplace=True)
df_distintivo_ambiental.to_csv('../bin/listado_distintivo_ambiental.csv', index=False)

conversion = {v: k for k, v in listado_distintivo_ambiental.items()}
df['id_distintivo_ambiental'] = df['id_distintivo_ambiental'].map(conversion)

In [21]:
listado_combustibles = {}
index = 1
for detalle in df['combustible'].unique():
    if pd.notnull(detalle): 
        listado_combustibles[index] = detalle
        index += 1
    
df_combustibles = pd.DataFrame.from_dict(listado_combustibles, orient='index', columns=['nombre_combustible']).reset_index()
df_combustibles.rename(columns={'index': 'id_combustible'}, inplace=True)
df_combustibles.to_csv('../bin/listado_combustibles.csv', index=False)

conversion = {v: k for k, v in listado_combustibles.items()}
df['id_combustible'] = df['combustible'].map(conversion)

df = df.drop(columns=['combustible'])

In [23]:
listado_tipos_cambio = {}
index = 1
for detalle in df['tipo_cambio'].unique():
    if pd.notnull(detalle): 
        listado_tipos_cambio[index] = detalle
        index += 1
    
df_tipos_cambio = pd.DataFrame.from_dict(listado_tipos_cambio, orient='index', columns=['nombre_tipo_cambio']).reset_index()
df_tipos_cambio.rename(columns={'index': 'id_tipo_cambio'}, inplace=True)
df_tipos_cambio.to_csv('../bin/listado_tipos_cambio.csv', index=False)

tipos_cambio_to_indices = {v: k for k, v in listado_tipos_cambio.items()}
df['id_tipo_cambio'] = df['tipo_cambio'].map(tipos_cambio_to_indices)

df = df.drop(columns=['tipo_cambio'])

In [25]:
listado_sobrealimentacion = {}
index = 1
for detalle in df['id_sobrealimentacion'].unique():
    if pd.notnull(detalle): 
        listado_sobrealimentacion[index] = detalle
        index += 1
    
df_sobrealimentacion = pd.DataFrame.from_dict(listado_sobrealimentacion, orient='index', columns=['nombre_sobrealimentacion']).reset_index()
df_sobrealimentacion.rename(columns={'index': 'id_sobrealimentacion'}, inplace=True)
df_sobrealimentacion.to_csv('../bin/listado_sobrealimentacion.csv', index=False)

conversion = {v: k for k, v in listado_sobrealimentacion.items()}
df['id_sobrealimentacion'] = df['id_sobrealimentacion'].map(conversion)

In [27]:
listado_traccion = {}
index = 1
for detalle in df['id_traccion'].unique():
    if pd.notnull(detalle): 
        listado_traccion[index] = detalle
        index += 1
    
df_traccion = pd.DataFrame.from_dict(listado_traccion, orient='index', columns=['nombre_traccion']).reset_index()
df_traccion.rename(columns={'index': 'id_traccion'}, inplace=True)
df_traccion.to_csv('../bin/listado_traccion.csv', index=False)

conversion = {v: k for k, v in listado_traccion.items()}
df['id_traccion'] = df['id_traccion'].map(conversion)

In [29]:
directorio_pkl = "../bin/imagenes_coches/"
os.makedirs(directorio_pkl, exist_ok=True)

archivos_pkl = sorted([f for f in os.listdir(directorio_pkl) if f.endswith(".pkl")])
dfs = [pd.read_pickle(os.path.join(directorio_pkl, archivo)) for archivo in archivos_pkl]

df_concatenado = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(columns=["link_anuncio", "foto_binaria"])

df_concatenado = df_concatenado.reset_index(drop=True).reset_index()
df_concatenado = df_concatenado.rename(columns={"index": "id_coche"})
df_concatenado["id_coche"] += 1

diccionario_id_coche = df_concatenado.set_index("id_coche")["link_anuncio"].to_dict()

conversion = {v: k for k, v in diccionario_id_coche.items()}
df['id_coche'] = df['link_coche'].map(conversion)
df = df.drop(columns=['link_coche'])

In [31]:
df.head()

,id_provincia,fecha_extraccion,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,cilindrada,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo,id_combustible,id_tipo_cambio,id_coche
0,1.0,25/11/2024,93000.0,1,1.0,12.0,20950.0,29044.0,4053.0,1751.0,1461.0,355.0,5.0,2548.0,1855.0,5.0,5.8,4.9,7.7,40.0,1984.0,1.0,6.0,147.0,200.0,320.0,237.0,6.7,10,2019,138.0,4.0,1.0,66.0,1.0,1.0,1,1
1,1.0,25/11/2024,236945.0,2,2.0,NaN,4000.0,15660.0,3999.0,1737.0,1488.0,285.0,5.0,2511.0,1695.0,5.0,4.6,3.8,5.9,45.0,1248.0,1.0,6.0,66.0,90.0,200.0,172.0,12.7,7,2009,127.0,4.0,1.0,52.0,2.0,2.0,2,2
2,2.0,25/11/2024,44300.0,3,3.0,12.0,15900.0,25030.0,4670.0,1814.0,1458.0,460.0,5.0,2680.0,1878.0,5.0,5.7,4.6,7.4,50.0,1395.0,1.0,6.0,81.0,110.0,200.0,195.0,10.9,8,2017,101.0,4.0,1.0,59.0,3.0,3.0,2,3
3,2.0,25/11/2024,10600.0,3,1.0,12.0,24900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN,NaN,3,2024,NaN,NaN,NaN,59.0,4.0,1.0,1,4
4,2.0,25/11/2024,13673.0,3,4.0,12.0,33500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204.0,NaN,NaN,NaN,2,2024,NaN,NaN,NaN,59.0,3.0,4.0,1,5


In [33]:
df.to_csv('../bin/prestaciones.csv', index=False)